In [ ]:
import pyfits
from scipy.ndimage import gaussian_filter
import scipy.optimize as opt
import numpy as np
import pylab as plt
import rydlab
import pandas as pd
import os
import copy
from lmfit import Model
from lmfit.models import LorentzianModel
from lmfit.models import ExponentialModel
from lmfit.models import ConstantModel
from lmfit import Parameters, minimize, report_fit, Model
from matplotlib.colors import LinearSegmentedColormap, to_rgb
import seaborn as sns
sns.set_style("dark")
#sns.set_style("darkgrid")
import matplotlib as mpl
import uncertainties as unc  
import uncertainties.unumpy as unumpy  
from uncertainties import ufloat

pd.options.display.max_colwidth = 120
mpl.rc('image', cmap='afmhot')
path= '/home/qd/Schreibtisch/Data/'
file_date = '2019-07-17'
folders = rydlab.analyze_folder(path,filter=False) 
slicer = (slice(15,95),slice(40,320))
binning_scale = 1
folders

In [ ]:
path=folders.Name[2]

os.chdir(path)

variables = np.loadtxt(file_date+'_variables.dat' )[:,1]

N = len(variables)

print(os.getcwd())

def params_to_dict(params):
    return {key: params[key].value for key in params.keys()}


class Fit2d:
    colors = [(1, 1, 1), to_rgb("#5597C6"), to_rgb("#60BD68"), to_rgb("#FAA43A"), to_rgb("#d37096")]

    c_map_name = 'my_list'
    cm = LinearSegmentedColormap.from_list(c_map_name, colors, N=200)

    @staticmethod
    def _function(x, y):
        pass
    
    def __init__(self, data, x=None, y=None, params=None):
        self.data = data
        if x is None or y is None:
            self.x, self.y = self.get_mesh()

        if params is None:
            model = Model(self._function)
            params = model.make_params()

        self.params = params
        self.fit_object = None

    def fit_data(self, method='LeastSq'):
        fit_object = minimize(self.residuals, self.params, method=method)
        self.fit_object = fit_object
        self.params = fit_object.params

    def residuals(self, p):
        return self.data - self._function([self.x, self.y], **params_to_dict(p))

    def plot(self, ax):
        ax.imshow(self.data, cmap=self.cm)
        ax.contour(self.x, self.y, self._function([self.x, self.y], **params_to_dict(self.params)),
                   5, colors='w', linewidths=0.5)

    def get_mesh(self):
        x = np.arange(0, np.shape(self.data)[1], 1)
        y = np.arange(0, np.shape(self.data)[0], 1)
        return np.meshgrid(x, y)

    def report(self):
        print(report_fit(self.fit_object))


class Fit2dGaussian(Fit2d):
    def __init__(self, data, x=None, y=None, params=None):
        super().__init__(data, x, y, params)

    @staticmethod
    def _function(args, amp=1, cen_x=250, cen_y=50, sig_x=50, sig_y=10, offset=0):
        x, y = args
        return amp * np.exp(-(((cen_x - x) / sig_x) ** 2 + ((cen_y - y) / sig_y) ** 2) / 2.0) + offset


class Fit2d2Gaussian(Fit2d):
    def __init__(self, data, x=None, y=None, params=None):
        super().__init__(data, x, y, params)

    @staticmethod
    def _function(args, amp=1, cen_x=250, cen_y=50, sig_x=50, sig_y=10,
                  amp2=1, cen_x2=250, cen_y2=50, sig_x2=50, sig_y2=10,
                  offset=0):
        x, y = args
        return (amp * np.exp(-(((cen_x - x) / sig_x) ** 2 + ((cen_y - y) / sig_y) ** 2) / 2.0)
                + amp2 * np.exp(-(((cen_x2 - x) / sig_x2) ** 2 + ((cen_y2 - y) / sig_y2) ** 2) / 2.0)
                + offset)


def fitsopen(n,bg):
    if n<10:
        hdulist = pyfits.open(file_date+str("_")+str(0)+str(n)+'.fts')
    else:
        hdulist = pyfits.open(file_date+str("_")+str(n)+'.fts')

    data=np.zeros((90,400))

    for y in range(10,100):
        for x in range(10,410):
            data[y-10,x-10]=-np.log((hdulist[0].data[0,y,x])/(hdulist[0].data[1,y,x]))

    hdulist.close()
    return data-bg

def fitsopen_bg(n,bg):
    hdulist = pyfits.open(file_date+str("_")+str(n).zfill(2)+'.fts')
    images = hdulist[0].data
    absorb = images[0]
    no_absorb = images[1]
    div = (absorb-bg)/(no_absorb-bg)
    
    div = div[slicer]
    div = -np.log(div)
    div = np.nan_to_num(div)
    return div

def fitsopen_std(n):
    os.chdir(path+'/std')
    hdulist = pyfits.open(file_date+str("_")+str(n).zfill(3)+'.fts')
    images = hdulist[0].data
    lightnoise=images[1][slicer]
    atomnoise=images[0][slicer]
    absorb_weights = 1/images[0]
    #no_absorb_weights = 1/images[1]
    #div = (absorb-bg)/(no_absorb-bg)
    #absorb_weights = absorb_weights[slicer]
    #absorb_weights = -np.log(absorb_weights)
    #absorb_weights = np.nan_to_num(absorb_weights)
    os.chdir(path)
    return atomnoise,lightnoise

def twoD_Gaussian(xy_mesh, amplitude, xo, yo, sigma_x, sigma_y, theta, offset):
    (x, y) = xy_mesh
    xo = float(xo)
    yo = float(yo)
    a = (np.cos(theta)**2)/(2*sigma_x**2) + (np.sin(theta)**2)/(2*sigma_y**2)
    b = -(np.sin(2*theta))/(4*sigma_x**2) + (np.sin(2*theta))/(4*sigma_y**2)
    c = (np.sin(theta)**2)/(2*sigma_x**2) + (np.cos(theta)**2)/(2*sigma_y**2)
    g = offset + amplitude*np.exp( - (a*((x-xo)**2) + 2*b*(x-xo)*(y-yo)
                        + c*((y-yo)**2)))
    return g.flatten()


def two_Gaussian(xy_mesh, amplitude1, xo1, yo1, sigma_x1, sigma_y1, theta1, amplitude2, xo2, yo2, sigma_x2, sigma_y2, theta2, offset):
    (x, y) = xy_mesh
    xo1 = float(xo1)
    yo1 = float(yo1)
    xo2 = float(xo2)
    yo2 = float(yo2)

    a1 = (np.cos(theta1)**2)/(2*sigma_x1**2) + (np.sin(theta1)**2)/(2*sigma_y1**2)
    b1 = -(np.sin(2*theta1))/(4*sigma_x1**2) + (np.sin(2*theta1))/(4*sigma_y1**2)
    c1 = (np.sin(theta1)**2)/(2*sigma_x1**2) + (np.cos(theta1)**2)/(2*sigma_y1**2)
    a2 = (np.cos(theta2)**2)/(2*sigma_x2**2) + (np.sin(theta2)**2)/(2*sigma_y2**2)
    b2 = -(np.sin(2*theta2))/(4*sigma_x2**2) + (np.sin(2*theta2))/(4*sigma_y2**2)
    c2 = (np.sin(theta2)**2)/(2*sigma_x2**2) + (np.cos(theta2)**2)/(2*sigma_y2**2)

    g = offset + amplitude1*np.exp( - (a1*((x-xo1)**2) + 2*b1*(x-xo1)*(y-yo1) + c1*((y-yo1)**2))) + amplitude2*np.exp( - (a2*((x-xo2)**2) + 2*b2*(x-xo2)*(y-yo2) + c2*((y-yo2)**2)))

    return g.flatten()


def make_background(N):
    list_bg=list()
    for n in range(0,N):
        hdulist = pyfits.open(file_date+str("_")+str(n).zfill(2)+'.fts')
        list_bg.append(hdulist[0].data[2])
        hdulist.close()
    bg_mean = np.array(list_bg).mean(axis=0)
    bg_std = np.array(list_bg).std(axis=0)
    return bg_mean,bg_std
        



def LZ(x,amplitude,W,x0,c):
    return amplitude*np.exp(2*np.pi*W**2/(x-x0))+c

In [ ]:
results=list()

model_twoG = Model(two_Gaussian)

params = Parameters()

params = model_twoG.make_params()


'''
    amplitude1:  0.51301272 (init = 0.1)
    xo1:         35.5498681 (init = 30)
    yo1:         129.827169 (init = 120)
    sigma_x1:    31.6873916 (init = 43)
    sigma_y1:    66.6836446 (init = 80)
    theta1:      8.2787e-04 (init = 0)
    amplitude2:  0 (fixed)
    xo2:         38.0000000 (init = 38)
    yo2:         133.000000 (init = 133)
    sigma_x2:    2.00000000 (init = 2)
    sigma_y2:    2.00000000 == 'sigma_x2'
'''

# cloud distribution
params.add('amplitude1',value=0.1)
params.add('yo1',value=130*binning_scale,min=100*binning_scale,max=200*binning_scale)
params.add('xo1',value=35*binning_scale,min=20*binning_scale,max=60*binning_scale)
params.add('sigma_x1',value=31*binning_scale,min=10*binning_scale,max=100*binning_scale)
params.add('sigma_y1',value=66*binning_scale,min=10*binning_scale,max=100*binning_scale)
params.add('theta1',value=0,min=-np.pi/10,max=np.pi/10)

# EIT/Autler-Townes-dip
params.add('amplitude2',value=0.00,min=-1,max=1,vary=True)
params.add('yo2',value=143*binning_scale,min=130*binning_scale,max=156*binning_scale,vary=True)
params.add('xo2',value=38*binning_scale,min=30*binning_scale,max=45*binning_scale,vary=True)
params.add('sigma_x2',value=2*binning_scale,min=2*binning_scale,max=10*binning_scale,vary=True)
#params.add('sigma_y2',value=10,min=3,max=15)
params.add('sigma_y2',expr='sigma_x2',vary=False)
params.add('theta2',value=0,min=0,max=np.pi,vary=False)

# offset
params.add('offset',value=0,vary=False)
bg_mean,bg_std = make_background(N)

model=model_twoG


for n in range(0,len(variables)):
    bg,std= make_background(N)
    image = fitsopen_bg(n,bg)
    image_weights=fitsopen_std(n)[0]
    image_weights=1/image_weights
    image_weights=image_weights/np.mean(image_weights)
    #image_weights=gaussian_filter(image_weights, 1, order=0, output=None, mode='nearest', cval=0.0, truncate=4.0)
    #image =  gaussian_filter(image, 1, order=0, output=None, mode='nearest', cval=0.0, truncate=4.0)
    shape = image.shape
    x,y = np.mgrid[0:shape[0],0:shape[1]]
    image_flat=image.flatten() 
    weights_flat=image_weights.flatten()
    out = model.fit(image_flat,params,xy_mesh=(x,y))##method='Powel')
    fig,ax = plt.subplots(4,1,figsize=(15,15))
    results.append(copy.deepcopy(out))
    #params = out.params
    #out.params.pretty_print()
    print(out.fit_report())
    #print(out.success)
    vmax = 0.4
    ax[0].set_title('Image Number '+str(n))
    ax[0].imshow(image, origin='bottom',vmin=0, vmax=vmax)
    fig.colorbar(ax[0].imshow(image, origin='bottom',vmin=0, vmax=vmax),ax=ax[0])
    ax[1].set_title('Fit')
    ax[1].imshow(out.best_fit.reshape(shape),origin='bottom',vmin=0, vmax=vmax)
    fig.colorbar(ax[1].imshow(out.best_fit.reshape(shape),origin='bottom',vmin=0, vmax=vmax),ax=ax[1])
    ax[2].set_title('Residual')
    ax[2].imshow((image-out.best_fit.reshape(shape))/out.best_fit.reshape(shape),origin='bottom',vmin=-1, vmax=1, cmap='coolwarm')
    fig.colorbar(ax[2].imshow((image-out.best_fit.reshape(shape))/out.best_fit.reshape(shape),origin='bottom',vmin=-1, vmax=1, cmap='coolwarm') ,ax=ax[2])   
    ax[3].set_title('Noise Map')
    ax[3].imshow(1/image_weights,origin='bottom',vmin=0.5, vmax=1.5, cmap='afmhot')
    fig.colorbar(ax[3].imshow(1/image_weights,origin='bottom',vmin=0.5, vmax=1.5, cmap='afmhot'), ax=ax[3])

    plt.show()

In [ ]:
Twolevel = [model_twoG.eval(r.params,xy_mesh=(r.params['xo2'].value,r.params['yo2'].value))[0]-r.params['amplitude2'].value  for r in results]
#Twolevel = [r.params['amplitude1'].value  for r in results]
#Twolevel= [r.eval(r.params,xy_mesh=(r.params['xo1'].value,r.params['yo1'].value))[0] for r in results]
Threelevel = [r.eval(r.params,xy_mesh=(r.params['xo2'].value,r.params['yo2'].value))[0] for r in results]
ThreeOverTwo=[r.eval(r.params,xy_mesh=(r.params['xo2'].value,r.params['yo2'].value))[0] /(model_twoG.eval(r.params,xy_mesh=(r.params['xo2'].value,r.params['yo2'].value))[0]-r.params['amplitude2'].value)  for r in results]
error2lvl= [r.eval_uncertainty(r.params,xy_mesh=(r.params['xo1'].value,r.params['yo1'].value))[0] for r in results]
error3lvl= [r.eval_uncertainty(r.params,xy_mesh=(r.params['xo2'].value,r.params['yo2'].value))[0] for r in results]
weight2lvl= [r.redchi*2.6*10**4 for r in results]
weight3lvl= [1/(1.5*r.eval_uncertainty(r.params,xy_mesh=(r.params['xo2'].value,r.params['yo2'].value))[0]) for r in results]

def TwoLvlTrans(Dp,Dp0,g31,mediumlength,wavelength,density,redlw,Wp):
    """
    prefactor in units of MHz*m^3
    density is in units of 10^15 1/m^3
    detunings,rabi-frequencys and Decay rates in MHz
    """  
    Wc=0
    g21=0
    Dp=Dp-Dp0
    prefactor=6.827*10**(-4)
    l=g31/2/( Dp**2 + (g31/2)**2 )
    l1=np.imag(g31*(-2*Dp+1.j*(g31+redlw))/(2 * Wp**2 *(g31+redlw)/g31 + (g31+redlw)**2 +4*Dp**2))/np.pi
    l2=(8* Dp**2 *g31) / abs((g31+1.j*2*Dp)*(1.j*2*Dp))**2
    return 2*mediumlength*(2*np.pi/wavelength)*prefactor*density*l

def ThreeLvlTrans(Dp,Dp0,Dc,g31,g21,Wc,density,wavelength,mediumlength):
    """
    Imaginary Part of the first order susceptibility
    in the ladder scheme:
    index 1 ~ ground state
    index 2 ~ Rydberg state
    index 3 ~ intermediate state
    prefactor in units of MHz*m^3
    density is in units of 10^15 1/m^3
    detunings,rabi-frequencys and Decay rates in MHz
    """  
    Dp=Dp-Dp0
    d = Dp-Dc
    prefactor=6.827*10**(-4)
    lside=0#(4*d*(Wc**2 - 4*d*Dp)-4*Dp*g21**2) / abs(Wc**2 + (g31+1.j*2*Dp)*(g21+1.j*2*d))**2
    rside=1.j*(8* d**2 *g31+2*g21*(Wc**2 + g21*g31)) / abs(Wc**2 + (g31+1.j*2*Dp)*(g21+1.j*2*d))**2
    
    return 2*mediumlength*(2*np.pi/wavelength)*np.imag(prefactor*density*(lside+rside))

modelL = Model(ThreeLvlTrans,independent_vars=['Dp'])
paramsL = modelL.make_params()
paramsL.add('wavelength',value=0.78,vary=False)
paramsL.add('mediumlength',value=25,vary=True)
paramsL.add('density',value=4,vary=False)
paramsL.add('redlw',value=0,vary=False)
paramsL.add('Wp',value=0,min=0.1,max=1,vary=False)


paramsL.add('g31',value=6.02,min=4,max=8,vary=False)
paramsL.add('Dp0',value=0,min=-1,max=1,vary=True)
paramsL.add('g21',value=10,min=0,max=2,vary=False)
paramsL.add('Wc',value=0,vary=False)

paramsL.add('Dc',value=0,min=-1,max=1,vary=False)

outL = modelL.fit(Twolevel,params=paramsL,weights=weight2lvl,Dp=variables)#,nan_policy='propagate')


Threelevel = [r.eval(r.params,xy_mesh=(r.params['xo2'].value,r.params['yo2'].value))[0] for r in results]



model_ThreeLvlTrans = Model(ThreeLvlTrans,independent_vars=['Dp'])
params_ThreeLvlTrans = model_ThreeLvlTrans.make_params()
params_ThreeLvlTrans.add('wavelength',value=outL.params['wavelength'].value,vary=False)
params_ThreeLvlTrans.add('mediumlength',value=outL.params['mediumlength'].value,vary=False)
params_ThreeLvlTrans.add('density',value=outL.params['density'].value,vary=False)

params_ThreeLvlTrans.add('g31',value=outL.params['g31'].value,min=5,max=7,vary=False)
params_ThreeLvlTrans.add('g21',value=0.0,min=0,max=1.3,vary=True)
params_ThreeLvlTrans.add('Wc',value=1.5,min=0.5,max=2.5,vary=True)

params_ThreeLvlTrans.add('Dc',value=0,min=-1,max=1,vary=True)
params_ThreeLvlTrans.add('Dp0',value=outL.params['Dp0'].value,min=-1,max=1,vary=False)
#params_ThreeLvlTrans.add('Dp',value=0.00)

out = model_ThreeLvlTrans.fit(Threelevel,params=params_ThreeLvlTrans,weights=weight2lvl,Dp=variables)#,nan_policy='omit')

v = np.linspace(min(variables),max(variables),200)

#y = [1-np.exp(-r.params['amplitude1'].value) for r in results]
#x = od_2
#y = (1-np.exp(-od_3))/(1-np.exp(-od_2)) 
#plt.errorbar(x,y,yerr=yerr,marker='o',linestyle='',markersize='4')

#plt.plot(v[50:150],model_ThreeLvlTrans.eval(params_ThreeLvlTrans,Dp=v)[50:150])
#plt.errorbar(variables[7:23],Twolevel[7:23],yerr=error[7:23],marker='o',linestyle='--',markersize='4')
#plt.errorbar(variables[7:23],Threelevel[7:23],yerr=error[7:23],marker='o',linestyle='--',markersize='4')
#plt.plot(v[50:150],model_ThreeLvlTrans.eval(out.params,Dp=v)[50:150])
#plt.plot(v[50:150],modelL.eval(outL.params,Dp=v)[50:150])

#plt.plot(v,model_ThreeLvlTrans.eval(params_ThreeLvlTrans,Dp=v))
#plt.plot(v,modelL.eval(paramsL,Dp=v))

In [ ]:
#plt.errorbar(variables,Twolevel,yerr=error2lvl,marker='o',linestyle='',markersize='4')
#plt.errorbar(variables,Threelevel,yerr=error3lvl,marker='o',linestyle='--',markersize='4')

plt.errorbar(Twolevel[1:16],ThreeOverTwo[1:16],yerr=error2lvl[1:16],marker='o',linestyle='',markersize='4')
#plt.errorbar(variables,Threelevel,yerr=error3lvl,marker='o',linestyle='--',markersize='4')
#plt.plot(v,model_ThreeLvlTrans.eval(out.params,Dp=v))
#plt.plot(v,modelL.eval(outL.params,Dp=v))
plt.errorbar(Twolevel[1:16],Twolevel[1:16]/(1+Twolevel[1:16]),yerr=error2lvl[1:16],marker='o',linestyle='',markersize='4')
plt.xlabel("Density")
plt.ylabel("Threelvl/Twolvl")
plt.savefig("DensityScan.png")
plt.show()

#plt.ylabel("2 lvl Weights")
#plt.plot(variables,weight2lvl)
#plt.show()s

#plt.ylabel("3 lvl Weights")
#plt.plot(variables,weight3lvl)
#plt.show()



#%notebook -e analysis.ipynb
#print(outL.fit_report())
#print(out.fit_report())
#print("2 LEVEL RESPONSE")
#outL.params.pretty_print()
#print("3 LEVEL RESPONSE")
#out.params.pretty_print()
print(np.mean(Threelevel))
print(np.std(Threelevel))
print(np.std(Threelevel)/np.mean(Threelevel))

In [ ]:
plt.xlabel("MWDuration")
plt.ylabel("Threelvl/Twolvl")


plt.errorbar(variables,Twolevel,yerr=error2lvl,marker='o',linestyle='',markersize='4')
plt.errorbar(variables,Threelevel,yerr=error3lvl,marker='o',linestyle='-',markersize='4')
plt.errorbar(variables,np.abs(ThreeOverTwo),yerr=error3lvl,marker='o',linestyle='--',markersize='4')
plt.savefig("DensityScan.png")
plt.show()

In [ ]:
ThreeDivTwolevel = [r.eval(r.params,xy_mesh=(r.params['xo2'].value,r.params['yo2'].value))[0]/
                    (model_twoG.eval(r.params,xy_mesh=(r.params['xo2'].value,r.params['yo2'].value))[0]-r.params['amplitude2'].value)  for r in results]
#Twolevel = [r.params['amplitude1'].value  for r in results]
#Twolevel= [r.eval(r.params,xy_mesh=(r.params['xo1'].value,r.params['yo1'].value))[0] for r in results]

v = np.linspace(min(variables),max(variables),200)

def ThreeOverTwo(Dint,g31,Wc,Wp,redlw):
    """
    Imaginary Part of the first order susceptibility
    in the ladder scheme:
    index 1 ~ ground state
    index 2 ~ Rydberg state
    index 3 ~ intermediate state
    prefactor in units of MHz*m^3
    density is in units of 10^15 1/m^3
    detunings,rabi-frequencys and Decay rates in MHz
    """  
    """
    prefactor in units of MHz*m^3
    density is in units of 10^15 1/m^3
    detunings,rabi-frequencys and Decay rates in MHz
    """ 
    
    l=4 * redlw * Dint**2 *(g31*redlw+2*Wp)
    l1=(Wp**2+Wc**2)*(g31*Wc**2+redlw*Wp**2)
    return l/(l+l1)

def ThreeTwoTrans(Dp,Dp0,Dc,g31,g21,Wc):
    """
    Imaginary Part of the first order susceptibility
    in the ladder scheme:
    index 1 ~ ground state
    index 2 ~ Rydberg state
    index 3 ~ intermediate state
    prefactor in units of MHz*m^3
    density is in units of 10^15 1/m^3
    detunings,rabi-frequencys and Decay rates in MHz
    """  
    Dp=Dp-Dp0
    d = Dp-Dc
    prefactor=6.827*10**(-4)
    lside=0#(4*d*(Wc**2 - 4*d*Dp)-4*Dp*g21**2) / abs(Wc**2 + (g31+1.j*2*Dp)*(g21+1.j*2*d))**2
    rside=(8* d**2 *g31+2*g21*(Wc**2 + g21*g31)) / abs(Wc**2 + (g31+1.j*2*Dp)*(g21+1.j*2*d))**2
    r1=(8* d**2 *g31+2*10*(10*g31)) / abs((g31+1.j*2*Dp)*(10+1.j*2*d))**2
    
    return rside/r1

    
model32 = Model(ThreeTwoTrans,independent_vars=['Dp'])
params32 = modelL.make_params()
params32.add('g31',value=6.02,min=4,max=8,vary=False)
params32.add('Dp0',value=0,min=-1,max=1,vary=False)
params32.add('g21',value=0.1,min=0,max=2,vary=False)
params32.add('Wc',value=1.5,vary=True)
params32.add('Dc',value=-0.14,min=-1,max=1,vary=False)

finalweights=np.ones(len(variables))
for n in range(11,19):
    finalweights[n]=0
print(finalweights)

#out32 = model32.fit(Twolevel,params=params32,weights=finalweights,Dp=variables)#,nan_policy='propagate')

#print(out32.fit_report())


plt.xlabel("Probe Detuning")
plt.ylabel("Chi_3lvl/Chi_2lvl")
plt.errorbar(variables,ThreeDivTwolevel,yerr=error3lvl,marker='o',linestyle='--',markersize='4')

##plt.plot(v,model32.eval(out32.params,Dp=v))

Dp=0
Dp0=0
Dc=-0.7
g31=6.02
g21=0.1
Wc=1.5
density=4
wavelength=0.78
mediumlength=24.6

C6=513000
g21=0.2
redlw=0.2
Wc=1.5
Wp=0.28
g31=6.02
N=4
density=0.004
rb=(2*C6/(g21*redlw+Wc**2))**(1/6)
vb=4/3*np.pi*rb**3
Dint=C6/rb**6*(density*vb/N)**2

print(Dint)

plt.plot(v,np.ones(200)*ThreeOverTwo(Dint,6.02,1.5,0.28,redlw))

plt.plot(v,np.ones(200)*ThreeLvlTrans(v,Dp0,Dc,g31,g21,Wc,density,wavelength,mediumlength)/ThreeLvlTrans(v,Dp0,Dc,g31,100,0,density,wavelength,mediumlength))



plt.plot(v,np.ones(200)*(0.0+4)/(1+4))


In [ ]:

v = np.linspace(-40,40,200)
plt.xlabel("Mean Field Detuning")
plt.ylabel("Chi_3lvl/Chi_2lvl")

plt.plot(v,ThreeOverTwo(v,6.02,1.5,0.28,0.1))



In [ ]:
def Pss(N,g21,Wc,C6,density,Wp,redlw):
    rb=(2*C6/(g21*redlw+Wc**2))**(1/6)
    vb=4/3*np.pi*rb**3
    Dint=C6/rb**6*(density*vb/N)**2
    rho_0=(Wp/Wc)**2
    rho=(1-ThreeOverTwo(Dint,g31,Wc,Wp,redlw))
    return rho*rho_0 * N/(rho*rho_0*N+1-rho*rho_0)

def Pss_no_int(N,Wc,Wp):
    rho_0=(Wp/Wc)**2
    return rho_0 * N/(rho_0*N+1-rho_0)




v = np.linspace(0,10,150)

plt.xlabel("N")
plt.ylabel("Pss")


#plt.plot(v,Pss_int(v,0.02,1.5,20,0.004,0.28,0.1))
plt.plot(v,Pss(v,0.02,1.5,513000,0.004,0.28,0.1))
plt.plot(v,0.5*Pss_no_int(v,1.5,0.28))



C6=513000
g21=0.2
redlw=0.2
Wc=1.5
Wp=0.28
g31=6.02
N=4
density=0.004
rb=(2*C6/(g21*redlw+Wc**2))**(1/6)
vb=4/3*np.pi*rb**3
Dint=C6/rb**6*(density*vb/N)**2
print(N)
print(ThreeOverTwo(Dint,g31,Wc,Wp,redlw))
